### INST326 OOP Project 03

Rename this notebook, replacing "_Assignment" with "_YourName"<br>
Insert Signature Block Here

#### Group 54: Christian Sorensen, Tamunosaki Danagogo, Marcos Alvarado
> INST326
> Project 3
> 11/1/2024
#### Honor Pledge
> I pledge that the work contained in this assignment is my own, and that I have complied with University and course policies on academic integrity and AI use.


### The Project
Everyone will do the same project this time. This is a group project, so you must work in your assigned groups. Include the link to your group's GitHub repository (one link per group). Use comments in your code to document your solution. If you need to write comments to the grader, add a markdown cell immediately above your code solution and add your comments there. Be sure to read and follow all the requirements and the Notebook Instructions at the bottom of this notebook. Your grade may depend on it!

#### 1. A Scheduling Program
>  My wife is responsible for scheduling caregivers for her 93 year-old mother. Currently she writes out the schedule on a monthly calendar and photocopies it for everyone. I want all of you to help me write a program to help her with scheduling. While this is a specific application, this program will be broadly useful and adaptable to any scheduling needs for small businesses, clubs, and more.

#### Requirements
>  Care is required 12 hours per day, 7 days a week. There are two shifts each day: 7:00 AM - 1:00 PM, and 1:00 PM to 7:00 PM. There are a total of 8 caregivers. Some are family members and some are paid. Each caregiver has their own availability for shifts that is generally the same from month to month, but there are exceptions for work, vacations, and other responsibilities. Your program should do the following:
> 1. Manage caregivers and their schedules. Attributes include: name, phone, email, pay rate, and hours.
> 2. Each caregiver should have their own availability schedule where they can indicate their availability for each shift. Availability categories are 'preferred', 'available' (default), and 'unavailable'.
> 3. Create a care schedule that covers AM and PM shifts and displays caregiver names on a calendar (see example). The schedule should accomodate caregivers' individual schedules and availability preferences. The python calendar module provides options for creating HTML calendars. Sample code for the HTML calendar is in the project folder.
> 4. Paid caregivers are paid weekly at $20/hr. Your program should calculate weekly pay based on assigned hours. Provide a separate pay report that lists weekly (gross: hours x rate) amounts to each caregiver, along with weekly and monthly totals. The report can be a text document, or presented in GUI or HTML format. 

#### Group Requirements
>  1. Your submitted project should follow OOP principles like abstraction, encapsulation, inheritance, and polymorphism as appropriate. Your program should use classes. 
>  2. Select a group leader who will host the group's project repository on their GitHub.
>  3. Create the group repository and add a main program document. See example.
>  4. Create branches off the main program for each group member, and assign part of the program to each member.
>  5. Each member should work on their branch.
>  6. When each member is finished, merge the branches back into the main program. You may use 'merge' or 'pull requests', your choice.
>  7. iterate and debug as necessary.

#### Working with HTML
> Since this is a course on python, not HTML, you are not expected to know HTML. Therefore, you may copy applicable portions of the sample code or use AI to write the HTML portions of your application. Ypu should write the main python code yourself.


#### What you need to turn in
>  This is a group project. There will be one submission per group. Your submission will be graded as a group.
>  1. Include your group number and the names of all group members in the signature block at the top of this notebook.
>  2. In the cell below, paste the link to your project repository. One link per group. The grader will review the activity and history provided by GitHub. To add a hyperlink to a Jupyter markdown cell, follow the instructions in the cell below.
>  3. Below the GitHub Repository Link cell is a code cell. Copy and paste your final program code into this cell.

#### GitHub Repository Link
> [Github Repo](https://github.com/chrissor-umd/INST326_Group_54)

In [ ]:
# Solution - enter your code solution below
import tkinter as tk
import datetime as dt # For handling data and time calculations
from datetime import time
import json # For saving and loading data

# Stuff used for the calendar
import calendar
from IPython.display import HTML


class ScheduleManager():
    def __init__(self):
        self.__load_json() # Loads necessary data for managing people and schedules


    #############################################################################################
    #######    SUPER AWESOME AND NOT ANNOYING JSON ENCODING STUFF I SPENT 6 HOURS ON :)    ######
    #############################################################################################

    @staticmethod
    def __json_people_hook(obj): # Converts JSON object into Person object when loading
        if "name" in obj:
            dict = obj
            name = dict["name"]
            #id = int(dict["_id"])
            phone = dict["phone"]
            email = dict["email"]
            pay_rate = dict["_Person__pay_rate"]
            shift_availability = dict["_Person__shift_availability"]
            shift_exceptions = dict["_Person__shift_exceptions"]
            person = Person(name, phone, email)
            person._Person__pay_rate = pay_rate
            person._Person__shift_availability = shift_availability
            person._Person__shift_exceptions = shift_exceptions
            return person
        return obj
    
    @staticmethod
    def __encode_person(obj):
        if isinstance(obj, Person):
            return obj.__dict__ # Converts a person into a serializable dictionary
        else:
            return super().default(obj)

    def __load_json(self):
        """Loads stored data from json files"""
        try:
            with open('people_data.json', 'r') as people_file:
                people_data = people_file.read()
                if len(people_data) > 0:
                    self.__people = json.loads(people_data, object_hook=self.__json_people_hook) # For some reason, I can only get this to work right by storing and loading it as a string
                else:
                    self.__people = {}
                print("Loaded people_data.json")
        except FileNotFoundError:
            with open('people_data.json', 'w') as people_file:
                self.__people = {}
                json.dump(self.__people, people_file, default=self.__encode_person)
                print("Created people_data.json")
        try:
            with open('shifts_data.json', 'r') as shifts_file:
                shifts = shifts_file.read()
                if len(shifts) > 0:
                    self.__shifts = json.loads(shifts)
                else:
                    self.__shifts = {'SUN':{}, # Each day is a dictionary that will contain each member with a shift as a key associated to a list of tuples with start-stop times for their shift
                                 'MON':{},
                                 'TUE':{},
                                 'WED':{},
                                 'THU':{},
                                 'FRI':{},
                                 'SAT':{}}
                print("Loaded shifts_data.json")
        except FileNotFoundError:
            with open('shifts_data.json', 'w') as shifts_file:
                self.__shifts = {'SUN':{}, # Each day is a dictionary that will contain each member with a shift as a key associated to a list of tuples with start-stop times for their shift
                                 'MON':{},
                                 'TUE':{},
                                 'WED':{},
                                 'THU':{},
                                 'FRI':{},
                                 'SAT':{}}
                json.dump(self.__shifts, shifts_file)
                print("Created shifts_data.json")

    def __save_json(self):
        """Handles data conversions for smooth json serialization, and loads data to file"""
        with open('people_data.json', 'w') as people_file:
            json_dump = json.dumps(self.__people, indent=4, default=self.__encode_person)
            people_file.write(json_dump)
        with open('shifts_data.json', 'w') as shifts_file:
            json_dump = json.dumps(self.__shifts, indent=4)
            shifts_file.write(json_dump)
            
    ################################################
    #######         END OF JSON STUFF         ######
    ################################################


    def add_person(self, name, phone=None, email=None):
        """Creates a new person object with a unique id"""
        person_id = name.upper()
        person = Person(name, phone, email) # Creates the person object
        if person_id in self.__people.keys():
            print("Person already exists")
            return person
        self.__people.update({person_id:person}) # Stores the new person in the people dictionary, with their name as the key
        print(f"Added {person.name}")
        self.__save_json()
        return person

    def remove_person(self, person_id):
        """Removes person from ScheduleManager"""
        person_id = person_id.upper()
        if person_id not in self.__people.keys(): # Checks if the person exists
            print("Person does not exist")
            return
        for shifts_day in self.__shifts.values(): # Get each day's list of shifts
            if person_id in shifts_day: # Remove the person's shifts from each day
                shifts_day.pop(person_id)
        self.__people.pop(person_id) # Removes person from list
        print(f"{person.name} removed")
        self.__save_json()

    def add_shift(self, person_id, day, start, end):
        """Schedules person for the allotted time, accounting for duplicate and overlapping shifts"""
        day, time_start, time_end = self.get_formatted_shift_input(day, start, end) # Ensures day and times are properly formatted
        if time_start >= time_end:
            print(f"Invalid time interval '{time_start.isoformat(timespec='minutes')}-{time_end.isoformat(timespec='minutes')}'")
            return
        shifts_dict = self.__shifts[day]
        if isinstance(person_id, Person):
            person_id = person_id.name
        person_id = person_id.upper()
        if person_id not in self.__people.keys(): # Check if person has been properly added
            print("Person does not exist")
            return
        person = self.__people[person_id]
        if person_id not in shifts_dict:
            shifts_dict.update({person_id:[[time_start.isoformat('minutes'), time_end.isoformat('minutes')]]}) # Adds user's name to dict with a list containing their new shift
            print(f"Person '{person.name}' added to shift '{time_start.isoformat(timespec='minutes')}-{time_end.isoformat(timespec='minutes')}'")
        else:
            shifts_list = shifts_dict[person_id]
            for shift in shifts_list:
                old_shift_start = time.fromisoformat(shift[0])
                old_shift_end = time.fromisoformat(shift[1])
                if time_start > old_shift_end or time_end < old_shift_start: # Check if the shift overlaps with a preexisting shift
                    continue
                elif time_start >= old_shift_start and time_end <= old_shift_end: # Checks if the start and stop times for the new shift are already within an existing shift
                    print(f"{person_id} already has shift in that range")
                    return
                else: # If there is a partial overlap, remove the existing shift and replace it with an extended combination of the old and new shift
                    shifts_list.remove(shift) # Removes the previous shift from the list to be replaced with the new one
                    time_start = min(time_start, old_shift_start) # Gets the earlier of the two shift start times
                    time_end = max(time_end, old_shift_end) # Gets the later of the two shift end times
                    break
            shifts_list.append([time_start.isoformat('minutes'),time_end.isoformat('minutes')]) # Appends the new shift as a list to the shifts dictionary for the given day
            print(f"Added {person.name} to shift")
            shifts_list.sort() # Sorts the list so the shifts are in chronological order
        self.__save_json()

    def remove_shift(self, person_id, day, start, end):
        """Removes person from the allotted time, removing or shrinking shifts where applicable"""
        day, time_start, time_end = self.get_formatted_shift_input(day, start, end) # Ensures day and time are properly formatted
        if time_start >= time_end:
            print(f"Invalid time interval '{time_start.isoformat(timespec='minutes')}-{time_end.isoformat(timespec='minutes')}'")
            return
        shifts_dict = self.__shifts[day]
        if isinstance(person_id, Person):
            person_id = person_id.name
        person_id = person_id.upper()
        if person_id not in self.__people.keys(): # Check if person has been properly added
            print("Person has not been added yet")
            return
        person = self.__people[person_id]
        if person_id in shifts_dict: # Check if the person has any shifts in the given day
            shift_remove_start = time_start # Rename variable for readability
            shift_remove_end = time_end
            shifts_list = shifts_dict[person_id]
            shift_add_queue = [] # Stores shifts to be added once iteration is complete, in cases of a shift being 'split'
            shift_remove_queue = [] # Stores shifts to be removed once iteration is complete, to avoid messy indexing problems
            shift_found = False
            for shift in shifts_list:
                old_shift_start = time.fromisoformat(shift[0]) # Convert the preexisting shift times to time. format for easy comparison
                old_shift_end = time.fromisoformat(shift[1])
                if shift_remove_start > old_shift_end or shift_remove_end < old_shift_start: # Check if the shift overlaps with a preexisting shift
                    continue
                elif shift_remove_start <= old_shift_start and shift_remove_end >= old_shift_end: # Checks if the start and stop times for the new shift are already within an existing shift
                    shift_found = True
                    shift_remove_queue.append(shift) # Remove any shift that is completely within the given time
                    continue
                else: # If there is a partial overlap, remove the existing shift and replace it with a shrunken version
                    shift_found = True
                    shift_remove_queue.append(shift) # Removes the previous shift from the list to be replaced with the new one(s)
                    if shift_remove_start > old_shift_start and shift_remove_end < old_shift_end: # If a shift is 'split' by the removal
                        print("split list")
                        shift_add_queue.append([old_shift_start.isoformat('minutes'), shift_remove_start.isoformat('minutes')]) # Adds the two separated shifts
                        shift_add_queue.append([shift_remove_end.isoformat('minutes'), old_shift_end.isoformat('minutes')])
                        break
                    if shift_remove_start > old_shift_start:
                        print("shrink shift end")
                        shift_add_queue.append([old_shift_start.isoformat('minutes'), shift_remove_start.isoformat('minutes')])
                        continue
                    elif shift_remove_end < old_shift_end:
                        print("shrink shift start")
                        shift_add_queue.append([shift_remove_end.isoformat('minutes'), old_shift_end.isoformat('minutes')])
                        continue
            for s in shift_remove_queue: # Removed queued shifts
                shifts_list.remove(s)
            for s in shift_add_queue: # Add queued shifts
                shifts_list.append(s)
            if len(shifts_list) == 0:
                shifts_dict.pop(person_id)
            if shift_found:
                print("Person removed from allotted time")
            else:
                print(f"Person not scheduled for any shifts within {time_start.isoformat('minutes')}-{time_start.isoformat('minutes')}")
            shifts_list.sort() # Sorts the list so the shifts are in chronological order
            self.__save_json()
        else:
            print(f"{person.name} not scheduled for any shifts during '{time_start.isoformat(timespec='minutes')}-{time_end.isoformat(timespec='minutes')}'")

    @staticmethod
    def get_formatted_shift_input(day, time_start, time_end): # This is a staticmethod so any class can call it easily
        """Returns a properly formatted and typechecked tuple of (day, time_start, time_end), raising error on invalid input"""
        if type(day) != str or type(time_start) != str or type(time_end) != str:
            raise Exception("Invalid format")
        day = day[0:3].upper() # Gets the uppercase forms of the first 3 letters of day, so inputs like 'Sunday' and 'sun' are both accepted
        if day not in {'SUN','MON','TUE','WED','THU','FRI','SAT'}:
            raise Exception("Invalid day")
        try:
            time_start = time.fromisoformat(time_start) # Ensure that the time is in a valid convertable format
            time_end = time.fromisoformat(time_end)
        except ValueError:
            print("Invalid time format")
            return
        for itime in (time_start, time_end): # Do some rounding and formatting for nicer and more predictable time intervals
            itime.replace(second=0)
            itime.replace(minute=(itime.minute - itime.minute%5)) # Round down to the nearest interval of 5 minutes
        return (day, time_start, time_end) # Returns a tuple containing the formatted and typechecked inputs

    def _debug_print_shifts(self): # DELETE THIS ONCE WE GET EVERYTHING ON THE HTML/GUI
        tab = "    "
        for day in self.__shifts:
            print(day)
            for person_id in self.__shifts[day]:
                print(f"{tab}{self.__people[person_id].name}:")
                for shift in self.__shifts[day][person_id]:
                    print(f"{tab*2}{shift[0]}-{shift[1]}")


class Person:
    def __init__(self, name, phone=None, email=None):
        self.name = str(name)
        self.phone = phone # Optional
        self.email = email # Optional
        self.__pay_rate = 0
        self.__shift_availability = {'SUN':{},
                                     'MON':{},
                                     'TUE':{},
                                     'WED':{},
                                     'THU':{},
                                     'FRI':{},
                                     'SAT':{}}
        
        self.__shift_exceptions = {'unavailable':{}, # After we set default values, overwrite shift availability based on this dict, depending on specific date
                                   'preferred':{}} 
    def __str__(self):
        """Returns name of person when object is converted to string"""
        return str(self.name)
    
    @property
    def pay_rate(self):
        """Read Only"""
        return self.__pay_rate

    def set_pay_rate(self, new_rate):
        """Sets the new rate, checking the type and rejecting negative numbers"""
        if type(new_rate) in {float, int}:
            if new_rate >= 0:
                self.__pay_rate = new_rate
            else:
                print("New rate must be positive number or zero")
        else:
            print("Invalid pay rate amount")

    @property
    def availability(self):
        """Read only"""
        return self.__shift_availability

    def change_availability(self, day, time, availability):
        """Changes the availability for the given day and time"""
        if availability not in {'preferred', 'available', 'unavailable'}: # Raise an error if we mess up and accidentally set a bad availability
            raise Exception("Invalid availability, please select only from: 'available', 'unavailable', 'preferred'")
        day, time_start, time_end = ScheduleManager.get_formatted_shift_input(day, time_start, time_end)
        shifts_dict = self.__shift_availability[day]

    def add_shift_exception(self, date, time, availability):
        """Adds an availaibility exeption for a specific date"""
        # My intention for this is that once we implement functionality for getting the current date and time, we can compare it to what's in the exception dict to overwrite the default values for that week
        pass

#Calendar stuff

class CalendarManager:
    def __init__(self):
        # Initialize HTMLCalendar
        html_cal = calendar.HTMLCalendar()
        year = 2024

        # creating the HTML content for the year 2024 calendar
        calendar_html = html_cal.formatyear(year)

        # Display the calendar as HTML in the notebook
        HTML(calendar_html)


if __name__ == "__main__":
    schedule = ScheduleManager()
    #person1 = schedule.add_person('Joe')
    #person2 = schedule.add_person('Bob')
    #person3 = schedule.add_person('Cliff')
    #schedule.remove_shift("Joe", 'SUN', "00:00", "12:45")
    schedule.remove_shift("joe", 'SUN', '10:30','10:40')
    #schedule.add_shift(person1, 'SUN', '10:15','10:50')
    #schedule.add_
    #shift(person3, 'SUN', )
    schedule._debug_print_shifts()

Loaded people_data.json
Loaded shifts_data.json
split list
Person removed from allotted time
SUN
    Joe:
        10:20-10:30
        10:40-10:50
MON
TUE
WED
THU
FRI
SAT


### Notebook Instructions
> Before turning in your notebook:
> 1. Make sure you have renamed the notebook file as instructed
> 2. Make sure you have included your signature block and that it is correct according to the instructions
> 3. comment your code as necessary
> 4. run all code cells and double check that they run correctly. If you can't get your code to run correctly and you want partial credit, add a note for the grader in a new markdown cell directly above your code solution.<br><br>
Turn in your notebook by uploading it to ELMS<br>
IF the exercises involve saved data files, put your notebook and the data file(s) in a zip folder and upload the zip folder to ELMS